<a href="https://colab.research.google.com/github/ArpanJainGithub/Chatbot-MentalHealth/blob/main/Phase_3_Modeling_and_Error_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Phase-3: Modeling and Error Analysis

Note- Not able to show the results in the report as BertClient has suddenly stopped working.

In [1]:
%tensorflow_version 1.x
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

TensorFlow 1.x selected.


In [2]:
#Referred the following video for unerstanding pupose: https://www.youtube.com/watch?v=ZxR38An5TQE&t=909s
#Referred the code from https://colab.research.google.com/drive/1BdTi2gqCPNeFUeNqJk83LHcjoqj4pOmw

###Using the BERT model to get the most similar question to the user query in the FAQ dataset.
BERT is choosen because-

1) It is state of the art technique for the text comparision.

2) Referred few resources (mentioned below) over internet which I felt was good, and most of them used BERT.
     
     a) https://medium.com/huggingface/how-to-build-a-state-of-the-art-conversational-ai-with-transfer-learning-2d818ac26313

     b) https://link.springer.com/chapter/10.1007/978-3-030-47426-3_19
     
     c) https://www.youtube.com/watch?v=ZxR38An5TQE&t=909s



Referred https://colab.research.google.com/drive/1BdTi2gqCPNeFUeNqJk83LHcjoqj4pOmw for code.

In [3]:
!pip install bert-serving-server

     |████████████████████████████████| 61 kB 243 kB/s 
  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7411 sha256=0b437c6a6abf6faf5daf51966f23ccfcf0ef0918b681453b4d65c7f1c06733d1
  Stored in directory: /root/.cache/pip/wheels/6e/f8/83/534c52482d6da64622ddbf72cd93c35d2ef2881b78fd08ff0c
Successfully built GPUtil


In [4]:
!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip; unzip uncased_L-12_H-768_A-12.zip

--2022-03-13 12:19:51--  https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.129.128, 209.85.234.128, 172.253.119.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.129.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 407727028 (389M) [application/zip]
Saving to: ‘uncased_L-12_H-768_A-12.zip’

uncased_L-12_H-768_ 100%[===================>] 388.84M   168MB/s    in 2.3s    

2022-03-13 12:19:54 (168 MB/s) - ‘uncased_L-12_H-768_A-12.zip’ saved [407727028/407727028]

Archive:  uncased_L-12_H-768_A-12.zip
   creating: uncased_L-12_H-768_A-12/
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: uncased_L-12_H-768_A-12/vocab.txt  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: uncased_L-12_H-768_A-12/bert_config.json  


In [5]:
!nohup bert-serving-start -max_seq_len=128 -model_dir=uncased_L-12_H-768_A-12 > out.file 2>&1 &

In [6]:
!pip install bert-serving-client

In [7]:
from bert_serving.client import BertClient
bc = BertClient()

KeyboardInterrupt: ignored

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
#Load Mental_Health_FAQ.csv into a pandas dataframe.
url= 'https://raw.githubusercontent.com/ArpanJainGithub/Chatbot-MentalHealth/main/Mental_Health_FAQ.csv'
mentalHealthFaq = pd.read_csv(url,encoding='unicode_escape')

In [ ]:
#Storing all the questions of the dataset in a list
quesCount = mentalHealthFaq['Questions'].shape[0]
questionList = []
for i in range(quesCount):
  questionList.append(mentalHealthFaq['Questions'].iloc[i])


In [ ]:
#Getting the Bert embeddings for all the question in the dataframe.
questionsBertEmbedding = []
for i in range(quesCount):
  questionsBertEmbedding.append(bc.encode([mentalHealthFaq['Questions'].iloc[i]]))
len(questionsBertEmbedding)

In [ ]:
#Function to fetch the similar question index for a given user-question

#Things that needs to be checked in this function. 1) What in case there exists multiple values having the max cosine-similarity. Whose index will this return?
def calculateCosineSimilarities(datasetEmbeddings, userQueryEmbeddings):
  cosineSimilarityList = []
  for i in range(len(datasetEmbeddings)):
    sim = cosine_similarity(datasetEmbeddings[i][0].reshape(1,-1), userQueryEmbeddings[0].reshape(1,-1))[0][0]
    cosineSimilarityList.append(sim)  
  return cosineSimilarityList

###Evaluating the model

Test Questions

Q1- What is mental illness?

Q2- What are the symptoms of mental illness?

Q3- How mental illness is caused and what is the treatment for that?

Q4- Is there any way to know if I am suffering from mental illness?

Q5- Is it possible to get some self-help material for depression?

In [ ]:
#Testing the model-
testQuestion1 = "What is mental illness?"
testQuestion2 = "What are the symptoms of mental illness?"
testQuestion3 = "How mental illness is caused and what is the treatment for that?"
testQuestion4 = "Is there any way to know if I am suffering from mental illness?"
testQuestion5 = "Is it possible to get some self-help material for depression?"
testQuestionBertEmbedding1 = bc.encode([testQuestion1])
testQuestionBertEmbedding2 = bc.encode([testQuestion2])
testQuestionBertEmbedding3 = bc.encode([testQuestion3])
testQuestionBertEmbedding4 = bc.encode([testQuestion4])
testQuestionBertEmbedding5 = bc.encode([testQuestion5])


Check the most similar question in the dataset for the testQuestion1.

In [ ]:
#Q1-Result
similarityQuestion1 = calculateCosineSimilarities(questionsBertEmbedding, testQuestionBertEmbedding1)
similarityQuestion1Index = similarityQuestion1.index(max(similarityQuestion1))
print("Index: "+ str(similarityQuestion1Index))
print("Similarity value: "+ str(similarityQuestion1[similarityQuestion1Index]))
print(questionList[similarityQuestion1Index])

The result for Q1 is wrong. There exists a more similar question in the FAQ dataset than what has been given by the bert model. Lets check what is the similarity score for the actually most similar question.

In [ ]:
#The result for Q1 is wrong. There exists a more similar question in the FAQ dataset than what has been given by the bert model. Lets check what is the 
#similarity score for that question.
print("Index: "+ str(0))
print("Similarity value: "+ str(similarityQuestion1[0]))
print(questionList[0])

Let us check the similarity scores that are greater than the similarity score of the question that is actually most similar to the user query.

In [ ]:
#Lets check what are the top similarity scores for the TestQuestion1-
descendingsimilarityQuestion1 = sorted(similarityQuestion1, reverse=True)
for i in range(5):
  print("Similarity score: "+ str(descendingsimilarityQuestion1[i]) + " and the corresponding index is: "+ str(similarityQuestion1.index(descendingsimilarityQuestion1[i])) + " and corresponding question is: " + questionList[similarityQuestion1.index(descendingsimilarityQuestion1[i])])


The actual answer ranks 4 in the list. Not satisfactory result. The similar analysis needs to be done with all the test questions. Some more test questions can be added on need basis.

Evaluate the model using Q2.

In [ ]:
#Q2-Result
Question2CosineSimilarityList = calculateCosineSimilarities(questionsBertEmbedding, testQuestionBertEmbedding2)
Question2SimilarIndex = Question2CosineSimilarityList.index(max(Question2CosineSimilarityList))
print("Index: "+ str(Question2SimilarIndex))
print("Similarity value: "+ str(Question2CosineSimilarityList[Question2SimilarIndex]))
print(questionList[Question2SimilarIndex])

Model gave the right question from the dataset that is most similar to the user query. Now, lets check the top 5 questions that are most similar to the user query.

In [ ]:
#Lets check what are the top similarity scores for the TestQuestion2-
descendingQuestion2CosineSimilarityList = sorted(Question2CosineSimilarityList, reverse=True)
for i in range(5):
  print("Similarity score: "+ str(descendingQuestion2CosineSimilarityList[i]) + " and the corresponding index is: "+ str(Question2CosineSimilarityList.index(descendingQuestion2CosineSimilarityList[i])) + " and corresponding question is: " + questionList[Question2CosineSimilarityList.index(descendingQuestion2CosineSimilarityList[i])])


Question: What are positive and negative symptoms?
The above question should rank higher than the second and third question that are shown in the top 5 most similar questions to the TestQuestion2.

###Summary of the performance of the above model

1) Sometimes model is giving the accurate results and sometimes it is giving the approximate results. Overall, the results are not as satisfactory as they should be.



###What can be improved if more time is given

1) Right now model is using the BERT model for semantic search. This can be improved by trying the different models for semantic search like-

1.   BERT model trained on Mental Health data. **MentalBert** is pre-trained model on the Mental Health data. Refer the following links to implement that- https://huggingface.co/mental/mental-bert-base-uncased and https://huggingface.co/sentence-transformers/bert-base-nli-mean-tokens
